In [1]:
import pandas as pd
import numpy as np
import requests
import gmaps

import sys

# This is a file path unique to my laptop to get to the API key. I did not save the API key in the repo 
# for security reasons.

sys.path.insert(1, '/Users/symbioticpenguin/Documents')

from api_keys import g_key

In [2]:
cities_data = pd.read_csv("../output_data/city_data.csv")

locations = cities_data[["Latitude","Longitude"]]
humidity = cities_data['Humidity (%)']
cities_data.head()

,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind (mph)
0,caravelas,-17.71,-39.25,79.790,84,0,11.654457
1,sorland,67.67,12.69,30.470,68,75,16.105968
2,hobart,-42.88,147.33,53.744,54,40,17.224438
3,vaini,-21.20,-175.20,82.670,74,40,9.171454
4,karratha,-20.74,116.85,83.228,78,78,14.830912


In [3]:
gmaps.configure(api_key=g_key,)

figure = gmaps.figure(center = (0,0), zoom_level = 1.75)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=True, max_intensity=200,
                                 point_radius = 20)
figure.add_layer(heat_layer)

figure

Figure(layout=FigureLayout(height='420px'))

In [4]:
ideal_locations = cities_data[cities_data['Temperature (F)']>=70]
ideal_locations = ideal_locations[cities_data['Temperature (F)']<=80]
ideal_locations = ideal_locations[cities_data['Wind (mph)']<=10]
ideal_locations = ideal_locations[cities_data['Cloudiness (%)'] == 0]

ideal_locations = ideal_locations.reset_index(drop = True)

ideal_locations


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,City,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind (mph)
0,neuquen,-38.95,-68.06,75.470,23,0,5.816044
1,karachi,24.91,67.08,71.870,24,0,5.816044
2,santiago del estero,-27.80,-64.26,76.262,58,0,4.518619
3,kruisfontein,-34.00,24.73,77.306,64,0,2.706697
4,rehoboth,-23.32,17.08,77.216,25,0,4.719943
5,lashio,22.93,97.75,72.806,28,0,5.771305
6,colesberg,-30.72,25.10,78.242,35,0,7.583227
7,tura,25.52,90.22,74.606,30,0,3.623843


In [5]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target = "Hotel"
radius = 10000
target_type = "Hotel"

hotels = []
locations = []

for index, row in ideal_locations.iterrows():
    coordinates = f"{row.Latitude}, {row.Longitude}"
    
    params = {
    "location": coordinates,
    "keyword": target,
    "radius": radius,
    "type": target_type,
    "key": g_key}
    
    response = requests.get(base_url, params=params).json()
    #print(response['results'][0]['name'])
    hotels.append(response["results"][0]["name"])
    locations.append(response["results"][0]["vicinity"])
#print(locations)   

In [6]:
### For this part I couldn't figure out how to add multiple strings to the info_box_content to reference 
### 2 separate lists, i.e. hotels and the region.


coords = ideal_locations[["Latitude", "Longitude"]].astype(float)

hotel_layer = gmaps.symbol_layer(
    coords, fill_color='rgba(150, 0, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=4,
    info_box_content=[f"Hotel Name: {hotel}" for hotel in hotels]
)

figure.add_layer(hotel_layer)

figure

Figure(layout=FigureLayout(height='420px'))